In [49]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))
tf.reset_default_graph()
n_epochs = 10
batch_size = 128
learning_rate = 0.01
nb_classes=10
# Step 1: Read in data
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True) 


# Step 2
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, nb_classes])

# Step 3
W = tf.Variable(tf.random_normal([784, nb_classes]))
b = tf.Variable(tf.random_normal([nb_classes]))

w_h = init_weights([784, 625])
w_h2 = init_weights([625, 625])
w_o = init_weights([625, 10])

drop_input=0.8
drop_hidden=0.5

#Input Layer
with tf.name_scope('input'):
    X = tf.nn.dropout(X, drop_input)

with tf.name_scope('hidden_layers'):
#Hidden Layer
    h = tf.nn.relu(tf.matmul(X, w_h))
    h = tf.nn.dropout(h, drop_hidden)

#hidden layer2    
    h2 = tf.nn.relu(tf.matmul(h, w_h2))
    h2 = tf.nn.dropout(h2, drop_hidden)



# Step 4
with tf.name_scope('predictions'):
    logits  = tf.matmul(h2, w_o)
    hypothesis = tf.nn.softmax(tf.matmul(h2, w_o),name='predictions')
    tf.summary.histogram('predictions', hypothesis)

# Step 5:
#cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
with tf.name_scope('cost'):
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits), name='cost')
    tf.summary.scalar('cost', cost)
# Step 6:
with tf.name_scope('train'):
    optimizer = tf.train.RMSPropOptimizer(2.5, 0.9).minimize(cost)
    predict_op = tf.argmax(hypothesis, 1)

merged = tf.summary.merge_all()

training_epochs = 20

teX,teY = mnist.test.images, mnist.test.labels



with tf.Session() as sess:
    train_writer = tf.summary.FileWriter('mnist_logs/rms_001', sess.graph)


    start_time = time.time()
    sess.run(tf.global_variables_initializer())	
    n_batches = int(mnist.train.num_examples/batch_size)
    
    
    for i in range(n_epochs): 
        total_loss = 0

        for _ in range(n_batches):
            X_batch, Y_batch = mnist.train.next_batch(batch_size)
            # Run the optimizer and loss, store the loss value as loss_batch
            summary,loss_batch, _ = sess.run([merged,cost, optimizer], feed_dict={X: X_batch, Y: Y_batch})
            
            total_loss += loss_batch
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
        
        
        
        train_writer.add_summary(summary, i+1)
    
    print("Final Accuracy", np.mean(np.argmax(teY, axis=1) ==
                         sess.run(predict_op, feed_dict={X: teX})))
   
    #print('Total time: {0} seconds'.format(time.time() - start_time))


    


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Average loss epoch 0: 11320.5637458
Average loss epoch 1: 1779.71003327
Average loss epoch 2: 12.1161576951
Average loss epoch 3: 2.34802590161
Average loss epoch 4: 2.30258488655
Average loss epoch 5: 2.30258488655
Average loss epoch 6: 2.30258488655
Average loss epoch 7: 2.30258488655
Average loss epoch 8: 2.30258488655
Average loss epoch 9: 2.30258488655
('Final Accuracy', 0.098000000000000004)
